In [12]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import uniform, beta, norm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from scipy.stats import skew, kurtosis
from tqdm import tqdm

# Data Processing

In [13]:
def calculate_descriptive_statistics(data):
    """
    Calculate descriptive statistics for each column in the dataset.
    
    Parameters:
        data (pd.DataFrame): DataFrame with numerical data for each index.
    
    Returns:
        pd.DataFrame: DataFrame containing Mean, Std. Dev., Skew, and Kurtosis.
    """
    stats = {
        "Mean": data.mean(),
        "Std. Dev.": data.std(),
        "Skew": data.apply(skew),
        "Kurtosis": data.apply(lambda x: kurtosis(x, fisher=True))  # Fisher=True gives excess kurtosis
    }
    
    return pd.DataFrame(stats)

In [14]:
# List of yfinance-compatible tickers
tickers = [
    "SPY",      # S&P 500 ETF (large-cap U.S. equities)
    "IWM",      # iShares Russell 2000 ETF (small-cap U.S. equities)
    "QQQ",      # Nasdaq 100 ETF (tech-heavy U.S. equities)
    "IEF",      # iShares 7-10 Year Treasury Bond ETF (intermediate bonds)
    "TLT",      # iShares 20+ Year Treasury Bond ETF (long-term bonds)
    "BND",      # Vanguard Total Bond Market ETF (broad bond market)
    "VNQ",      # Vanguard Real Estate ETF (U.S. REITs)
    "GLD",      # SPDR Gold Shares (gold commodity)
    "DBC",      # Invesco DB Commodity Index Tracking Fund (broad commodities)
    "VTI"       # Vanguard Total Stock Market ETF (overall U.S. equities)
]

# Download monthly returns data for the last 14 years
start_date = "2011-11-01"
end_date = "2024-11-01"

# Fetch monthly data for each ticker
monthly_returns = {}
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, interval='1mo', progress=False)['Adj Close']
    returns = data.pct_change().dropna() * 100  # Calculate monthly returns
    monthly_returns[ticker] = returns

# Combine all into a single DataFrame
monthly_returns_df = pd.DataFrame(monthly_returns)
monthly_returns_df.index.name = "Date"

# Abbreviation mapping for tickers
abbreviation_mapping = {
    "SPY": "USE",     # Large-cap U.S. equities
    "IWM": "USSC",    # Small-cap U.S. equities
    "QQQ": "UST",     # Technology-focused U.S. equities
    "IEF": "USB",     # Intermediate-term U.S. bonds
    "TLT": "LTB",     # Long-term U.S. bonds
    "BND": "BB",      # Broad U.S. bond market
    "VNQ": "USR",     # U.S. REITs
    "GLD": "GC",      # Gold commodity
    "DBC": "BC",      # Broad commodities
    "VTI": "TSE"      # Total U.S. equities
}

# Rename columns based on the abbreviation mapping
monthly_returns_df.rename(columns=abbreviation_mapping, inplace=True)
monthly_returns_df

,USE,USSC,UST,USB,LTB,BB,USR,GC,BC,TSE
Date,,,,,,,,,,
2011-12-01,0.408015,0.027108,-0.993055,1.812880,3.124899,0.468406,3.701063,-10.662434,-2.894356,0.405998
2012-01-01,5.301067,7.668438,8.833362,1.239524,0.237975,1.602843,7.558280,11.395481,3.688525,5.667619
2012-02-01,4.340577,2.568956,6.410042,-1.277254,-2.829941,-0.202221,-1.150725,-2.964978,5.353938,4.218455
2012-03-01,2.766024,2.171514,4.874998,-1.573824,-4.224694,-0.497515,4.361363,-1.320834,-1.773539,2.627505
2012-04-01,-0.232274,-1.316574,-1.001722,2.499252,4.801574,1.003277,3.686116,-0.148032,-1.354158,-0.210375
...,...,...,...,...,...,...,...,...,...,...
2024-06-01,3.195099,-1.399620,6.301166,1.215946,1.825939,0.877783,0.624706,-0.134700,-0.171899,2.714648
2024-07-01,1.537427,10.643647,-1.521827,2.890084,3.654331,2.353104,9.281175,5.367196,-2.798104,2.252879
2024-08-01,2.336556,-1.688551,1.103867,1.349349,2.095666,1.453755,5.220651,2.092249,-2.081485,2.131556


In [15]:
# Calculate Descriptive Statistics
descriptive_stats = calculate_descriptive_statistics(monthly_returns_df)
descriptive_stats

,Mean,Std. Dev.,Skew,Kurtosis
USE,1.223233,4.144438,-0.444681,0.978387
USSC,0.971101,5.501756,-0.333031,1.637728
UST,1.598465,5.009285,-0.273878,0.221404
USB,0.117529,1.868574,0.037401,0.112869
LTB,0.131238,3.904732,0.319343,0.167585
BB,0.152360,1.392525,-0.028788,1.141847
USR,0.800214,5.064624,-0.412084,1.282603
GC,0.351601,4.363482,0.188758,-0.083944
BC,0.039117,4.790997,-0.398904,0.449247
TSE,1.201381,4.253102,-0.459499,1.213139


In [16]:
monthly_returns_df.describe()

,USE,USSC,UST,USB,LTB,BB,USR,GC,BC,TSE
count,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000
mean,1.223233,0.971101,1.598465,0.117529,0.131238,0.152360,0.800214,0.351601,0.039117,1.201381
std,4.144438,5.501756,5.009285,1.868574,3.904732,1.392525,5.064624,4.363482,4.790997,4.253102
min,-12.998732,-21.779559,-13.488935,-4.737245,-9.442612,-4.192765,-20.016045,-11.058839,-17.340184,-14.311355
25%,-0.802328,-1.986510,-1.502412,-1.020461,-2.557437,-0.549809,-2.524590,-2.507565,-2.981861,-0.945027
50%,1.701163,1.214503,1.996835,0.031522,-0.035009,0.112023,1.127501,-0.167167,-0.038801,1.734416
75%,3.479714,4.215610,4.816796,1.218858,2.267531,0.868345,3.693589,2.960073,3.455202,3.498233
max,13.361051,18.244171,15.218765,4.634434,11.045844,4.519331,13.191192,11.395481,10.197628,13.695173


In [17]:
monthly_returns_df.describe()

,USE,USSC,UST,USB,LTB,BB,USR,GC,BC,TSE
count,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000
mean,1.223233,0.971101,1.598465,0.117529,0.131238,0.152360,0.800214,0.351601,0.039117,1.201381
std,4.144438,5.501756,5.009285,1.868574,3.904732,1.392525,5.064624,4.363482,4.790997,4.253102
min,-12.998732,-21.779559,-13.488935,-4.737245,-9.442612,-4.192765,-20.016045,-11.058839,-17.340184,-14.311355
25%,-0.802328,-1.986510,-1.502412,-1.020461,-2.557437,-0.549809,-2.524590,-2.507565,-2.981861,-0.945027
50%,1.701163,1.214503,1.996835,0.031522,-0.035009,0.112023,1.127501,-0.167167,-0.038801,1.734416
75%,3.479714,4.215610,4.816796,1.218858,2.267531,0.868345,3.693589,2.960073,3.455202,3.498233
max,13.361051,18.244171,15.218765,4.634434,11.045844,4.519331,13.191192,11.395481,10.197628,13.695173


# Model Construction

## Base Model

In [18]:
class BaseModel:
    """
    Base class for all models, providing shared functionality, variables, parameters, and methods.
    """
    def __init__(self, observation, num_particles = 1_000_000, time_steps = 155):
        """Initialize base model parameters, variables, and shared functions."""
        self.observation = observation.values  # Y = returns.values
        self.L = num_particles  # Number of particles
        self.T = time_steps  # Number of time steps
        self.δ = 0.98
        self.α_shrink_factor = (3 * self.δ - 1) / (2 * self.δ)
        self.train = self.observation[:48]

        # Common variables
        self.variables = {
            "x": np.zeros((self.T, self.L)),  # Latent x
            "p_x": np.zeros((self.T, self.L)), # particles of latent x, shape (T, L)
            "x̄": np.zeros((self.T, self.L)),
            "p_x̄":np.zeros((self.T, self.L)),
            "ϕ_x":np.zeros((self.T, self.L)),
            "p_ϕ_x":np.zeros((self.T, self.L)),
            "σ_x": np.zeros((self.T, self.L)),
            "p_σ_x":np.zeros((self.T, self.L)),

            "α":np.zeros((self.T, self.L)),
            "y": np.zeros((self.T, self.L)),  # Estimated return
            "ξ": np.zeros((self.T, self.L)),
            "ϵ":np.zeros((self.T, self.L)),
        }
            
    @staticmethod
    def N(mean, variance, size=None):
        """Generate samples from a normal distribution."""
        return np.random.normal(mean, np.sqrt(variance), size)

    @staticmethod
    def U(start_point, end_point, size=None):
        """Generate samples from a uniform distribution."""
        return np.random.uniform(start_point, end_point, size)

    @staticmethod
    def B(α, β, size=None):
        """Generate samples from a beta distribution."""
        return np.random.beta(α, β, size)

    @staticmethod
    def SOSS_update(α, θ):
        """Perform second-order stochastic smoothing."""
        return BaseModel.N(α * θ + (1 - α) * np.mean(θ), (1 - α**2) * np.var(θ))

    @staticmethod
    def x_transition_function(x̄, ϕ_x, σ_x, ξ_t, x_t):
        """Transition function for variable x."""
        return x̄ + ϕ_x * (x_t - x̄) + σ_x * ξ_t

    @staticmethod
    def observation_function(μ_t, x_t, ϵ_t):
        """Observation function."""
        return μ_t + np.exp(x_t / 2) * ϵ_t

    @staticmethod
    def observation_likelihood(y, μ, x):
        """Calculate the observation likelihood."""
        σ_t = np.exp(x / 2)
        return (1 / (np.sqrt(2 * np.pi) * σ_t)) * np.exp(-((y - μ) ** 2) / (2 * σ_t**2))
    
    @staticmethod
    def compute_stats(data):
        median = np.median(data, axis=1)
        lower_percentile = np.percentile(data, 2.5, axis=1)
        upper_percentile = np.percentile(data, 97.5, axis=1)
        return median, lower_percentile, upper_percentile

## CMSV

In [19]:
class CMSVModel(BaseModel):
    """
    CMSV Model class.
    """
    def __init__(self, observation, num_particles, time_steps, β=0.8):
        super().__init__(observation, num_particles, time_steps)
        # Variables
        self.variables.update({
            "μ̄":np.zeros((self.T, self.L)),
            "p_μ̄":np.zeros((self.T, self.L)),
        })

        # Initialization
        self.variables["μ̄"][0] = self.U(-5, 5, self.L)
        self.variables["x̄"][0] = self.U(-1, 5, self.L)
        self.variables["ϕ_x"][0] = 2 * self.B(20, 1.5, self.L) - 1
        self.variables["σ_x"][0] = self.U(0, 2, self.L)
        
        self.variables["x"][0] = self.N(
            self.variables["x̄"][0], 
            (self.variables["σ_x"][0] ** 2) / (1 - self.variables["ϕ_x"][0] ** 2)
        )
        self.variables["ϵ"][0] = self.N(0, 1, self.L)
        self.variables["y"][0] = self.observation_function(
            self.variables["μ̄"][0], self.variables["x"][0], self.variables["ϵ"][0]
        )

    def particle_filtering(self):
        """Particle filtering process for state estimation."""
        for t in tqdm(range(1, self.T)):
            # SOSS update for prior estimates
            self.variables["p_x̄"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["x̄"][t - 1])
            self.variables["p_ϕ_x"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["ϕ_x"][t - 1])
            self.variables["p_σ_x"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["σ_x"][t - 1])
            self.variables["p_μ̄"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["μ̄"][t - 1])

            # Generate system noise and calculate particles for x
            self.variables["ξ"][t] = self.N(0, 1, self.L)
            self.variables["p_x"][t] = self.x_transition_function(
                self.variables["p_x̄"][t],
                self.variables["p_ϕ_x"][t],
                self.variables["p_σ_x"][t],
                self.variables["ξ"][t],
                self.variables["x"][t - 1],
            )

            # Generate observation noise and calculate expected observations
            self.variables["ϵ"][t] = self.N(0, 1, self.L)
            self.variables["y"][t] = self.observation_function(
                self.variables["p_μ̄"][t], self.variables["p_x"][t], self.variables["ϵ"][t]
            )

            # Calculate likelihood using Gaussian observation likelihood
            self.variables["α"][t] = self.observation_likelihood(self.observation[t], self.variables["p_μ̄"][t], self.variables["p_x"][t])

            # Normalize the likelihood to calculate weights
            weights = self.variables["α"][t]
            weights /= np.sum(self.variables["α"][t])

            # Resample particles based on weights
            cumulative_sum = np.cumsum(weights)
            positions = (np.arange(self.L) + np.random.uniform(0, 1)) / self.L
            sample_indices = np.searchsorted(cumulative_sum, positions)

            # Resample all variables
            self.variables["x"][t] = self.variables["p_x"][t, sample_indices]
            self.variables["x̄"][t] = self.variables["p_x̄"][t, sample_indices]
            self.variables["μ̄"][t] = self.variables["p_μ̄"][t, sample_indices]
            self.variables["ϕ_x"][t] = self.variables["p_ϕ_x"][t, sample_indices]
            self.variables["σ_x"][t] = self.variables["p_σ_x"][t, sample_indices]




## SMSV

In [20]:
class SMSVModel(BaseModel):
    """
    SMSV Model class.
    """
    def __init__(self, observation, num_particles, time_steps, β=0.8):
        super().__init__(observation, num_particles, time_steps)

        # Variables
        self.variables.update({
            'μ':np.zeros((self.T, self.L)),
            "p_μ":np.zeros((self.T, self.L)),
            "μ̄":np.zeros((self.T, self.L)),
            "p_μ̄":np.zeros((self.T, self.L)),
            "σ_μ": np.zeros((self.T, self.L)),
            "p_ϕ_μ":np.zeros((self.T, self.L)),
            "σ_μ":np.zeros((self.T, self.L)),
            "p_σ_μ":np.zeros((self.T, self.L)),
            "η": np.zeros((self.T, self.L)),
        })

        # Initialization
        self.variables["μ̄"][0] = self.U(-5, 5, self.L)
        self.variables["x̄"][0] = self.U(-1, 5, self.L)
        self.variables["ϕ_μ"][0] = self.U(0, 1, self.L)
        self.variables["σ_μ"][0] = self.U(0, np.std(self.train), self.L)
        self.variables["ϕ_x"][0] = 2 * self.B(20, 1.5, self.L) - 1
        self.variables["σ_x"][0] = self.U(0, 2, self.L)
        self.variables["x"][0] = self.N(
            self.variables["x̄"][0], 
            (self.variables["σ_x"][0] ** 2) / (1 - self.variables["ϕ_x"][0] ** 2)
        )
        self.variables["μ"][0] = self.N(
            self.variables["μ̄"][0], 
            (self.variables["σ_μ"][0] ** 2) / (1 - self.variables["σ_μ"][0] ** 2)
        )        
        
        self.variables["ϵ"][0] = self.N(0, 1, self.L)
        self.variables["y"][0] = self.observation_function(
            self.variables["μ"][0], self.variables["x"][0], self.variables["ϵ"][0]
        )

    # functions
    def μ_transition_function(μ̄, ϕ_μ, σ_μ, η_μ, μ_t):
        particles = μ̄ + 
        * (μ_t - μ̄) + σ_μ * η_μ
        return particles

    def particle_filtering(self):
        """Particle filtering process for state estimation."""
        for t in tqdm(range(1, self.T)):
            # SOSS update for prior estimates
            self.variables["p_x̄"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["x̄"][t - 1])
            self.variables["p_ϕ_x"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["ϕ_x"][t - 1])
            self.variables["p_σ_x"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["σ_x"][t - 1])
            self.variables["p_μ̄"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["μ̄"][t - 1])
            self.variables["p_σ_μ"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["σ_μ"][t - 1])
            self.variables["p_ϕ_μ"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["ϕ_μ"][t - 1])

            # Generate system noise and calculate particles for x
            self.variables["ξ"][t] = self.N(0, 1, self.L)
            self.variables["p_x"][t] = self.x_transition_function(
                self.variables["p_x̄"][t],
                self.variables["p_ϕ_x"][t],
                self.variables["p_σ_x"][t],
                self.variables["ξ"][t],
                self.variables["x"][t - 1],
            )

            # Generate system noise and calculate particles for μ
            self.variables["η"][t] = self.N(0, 1, self.L)
            self.variables["p_μ"][t] = self.μ_transition_function(
                self.variables["p_μ̄"][t],
                self.variables["p_ϕ_μ"][t],
                self.variables["p_σ_μ"][t],
                self.variables["η"][t],
                self.variables["μ"][t - 1],
            )

            # Generate observation noise and calculate expected observations
            self.variables["ϵ"][t] = self.N(0, 1, self.L)
            self.variables["y"][t] = self.observation_function(
                self.variables["p_μ"][t], self.variables["p_x"][t], self.variables["ϵ"][t]
            )

            # Calculate likelihood using Gaussian observation likelihood
            likelihood = self.observation_likelihood(self.observation[t], self.variables["p_μ"][t], self.variables["p_x"][t])
            self.variables["α"][t] = likelihood

            # Normalize the likelihood to calculate weights
            weights = self.variables["α"][t]
            weights /= np.sum(weights)

            # Resample particles based on weights
            cumulative_sum = np.cumsum(weights)
            positions = (np.arange(self.L) + np.random.uniform(0, 1)) / self.L
            sample_indices = np.searchsorted(cumulative_sum, positions)

            # Resample all variables
            self.variables["x"][t] = self.variables["p_x"][t, sample_indices]
            self.variables["μ"][t] = self.variables["p_μ"][t, sample_indices]
            self.variables["x̄"][t] = self.variables["p_x̄"][t, sample_indices]
            self.variables["μ̄"][t] = self.variables["p_μ̄"][t, sample_indices]
            self.variables["ϕ_x"][t] = self.variables["p_ϕ_x"][t, sample_indices]
            self.variables["σ_x"][t] = self.variables["p_σ_x"][t, sample_indices]
            self.variables["ϕ_μ"][t] = self.variables["p_ϕ_μ"][t, sample_indices]
            self.variables["σ_μ"][t] = self.variables["p_σ_μ"][t, sample_indices]


## SMSV_EMA

In [21]:
class SMSV_EMAModel(BaseModel):
    """
    SMSV_EMA Model class.
    """
    def __init__(self, observation, num_particles, time_steps, β=0.8):
        super().__init__(observation, num_particles, time_steps)
        # Parameter
        self.α_μ_parameter = np.full(self.L, np.mean(observation[:24]))
        self.ϕ_μ = np.full(self.L, 1 - self.β)# shape (L, 1)
        self.σ_μ = self.U(0, self.calculate_σ_μ(self.α_μ_parameter, self.observation, self.β), self.L)
        
        # Initialization
        self.variables["x̄"][0] = self.U(-1, 5, self.L)
        self.variables["ϕ_x"][0] = 2 * self.B(20, 1.5, self.L) - 1
        self.variables["σ_x"][0] = self.U(0, 2, self.L)
        
        self.variables["x"][0] = self.N(
            self.variables["x̄"][0], 
            (self.variables["σ_x"][0] ** 2) / (1 - self.variables["ϕ_x"][0] ** 2)
        )
        self.variables["μ"][0] = self.N(
            self.α_μ_parameter, 
            (self.variables["σ_μ"][0] ** 2) / (1 - self.variables["σ_μ"][0] ** 2)
        )   
        self.variables["ϵ"][0] = self.N(0, 1, self.L)
        self.variables["y"][0] = self.observation_function(
            self.variables["μ"][0], self.variables["x"][0], self.variables["ϵ"][0]
        )

    def μ_transition_function(y_t, ϕ_μ, μ_t):
        particles = y_t + ϕ_μ * (μ_t - y_t)
        return particles
    
    def calculate_σ_μ(α, Y, β):
        μ̄ = np.zeros(len(Y))
        μ̄[0] = α[0]
        for i in range(1, len(Y)):
            μ̄[i] = β * Y[i - 1] + (1 - β) * μ̄[i - 1]
        return np.std(μ̄)
    def particle_filtering(self):
        """Particle filtering process for state estimation."""
        for t in tqdm(range(1, self.T)):
            # SOSS update for prior estimates
            self.variables["p_x̄"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["x̄"][t - 1])
            self.variables["p_ϕ_x"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["ϕ_x"][t - 1])
            self.variables["p_σ_x"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["σ_x"][t - 1])
            self.variables["p_μ̄"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["μ̄"][t - 1])
            self.variables["p_σ_μ"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["σ_μ"][t - 1])
            self.variables["p_ϕ_μ"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["ϕ_μ"][t - 1])

            # Generate system noise and calculate particles for x
            self.variables["ξ"][t] = self.N(0, 1, self.L)
            self.variables["p_x"][t] = self.x_transition_function(
                self.variables["p_x̄"][t],
                self.variables["p_ϕ_x"][t],
                self.variables["p_σ_x"][t],
                self.variables["ξ"][t],
                self.variables["x"][t - 1],
            )

            # Generate system noise and calculate particles for μ
            self.variables["η"][t] = self.N(0, 1, self.L)
            self.variables["p_μ"][t] = self.μ_transition_function(
                self.variables["p_μ̄"][t],
                self.variables["p_ϕ_μ"][t],
                self.variables["p_σ_μ"][t],
                self.variables["η"][t],
                self.variables["μ"][t - 1],
            )

            # Generate observation noise and calculate expected observations
            self.variables["ϵ"][t] = self.N(0, 1, self.L)
            self.variables["y"][t] = self.observation_function(
                self.variables["p_μ"][t], self.variables["p_x"][t], self.variables["ϵ"][t]
            )

            # Calculate likelihood using Gaussian observation likelihood
            likelihood = self.observation_likelihood(self.observation[t], self.variables["p_μ"][t], self.variables["p_x"][t])
            self.variables["α"][t] = likelihood

            # Normalize the likelihood to calculate weights
            weights = self.variables["α"][t]
            weights /= np.sum(weights)

            # Resample particles based on weights
            cumulative_sum = np.cumsum(weights)
            positions = (np.arange(self.L) + np.random.uniform(0, 1)) / self.L
            sample_indices = np.searchsorted(cumulative_sum, positions)

            # Resample all variables
            self.variables["x"][t] = self.variables["p_x"][t, sample_indices]
            self.variables["μ"][t] = self.variables["p_μ"][t, sample_indices]
            self.variables["x̄"][t] = self.variables["p_x̄"][t, sample_indices]
            self.variables["μ̄"][t] = self.variables["p_μ̄"][t, sample_indices]
            self.variables["ϕ_x"][t] = self.variables["p_ϕ_x"][t, sample_indices]
            self.variables["σ_x"][t] = self.variables["p_σ_x"][t, sample_indices]
            self.variables["ϕ_μ"][t] = self.variables["p_ϕ_μ"][t, sample_indices]
            self.variables["σ_μ"][t] = self.variables["p_σ_μ"][t, sample_indices]

    

## SMSV_EMASO

In [37]:
class SMSV_EMASOModel(BaseModel):
    """
    SMSV_EMASO Model class.
    """
    def __init__(self, observation, num_particles, time_steps, σ_μ=1.0):
        super().__init__(observation, num_particles, time_steps)
        # Parameter
        self.α_μ_parameter = np.full(self.L, np.mean(observation[:24]))

        # Variables
        self.variables.update({
            'μ':np.zeros((self.T, self.L)),
            "p_μ":np.zeros((self.T, self.L)),
            "μ̄":np.zeros((self.T, self.L)),
            "p_μ̄":np.zeros((self.T, self.L)),
            "ϕ_μ": np.zeros((self.T, self.L)),
            "p_ϕ_μ":np.zeros((self.T, self.L)),
            "σ_μ":np.zeros((self.T, self.L)),
            "p_σ_μ":np.zeros((self.T, self.L)),
            "η": np.zeros((self.T, self.L)),
        })
        print(self.variables["ϕ_μ"])

        # Initialization
        self.variables["x̄"][0] = self.U(-1, 5, self.L)
        self.variables["ϕ_μ"][0] = self.U(0, 1, self.L)
        self.variables["ϕ_x"][0] = 2 * self.B(20, 1.5, self.L) - 1
        self.variables["σ_x"][0] = self.U(0, 2, self.L)
        
        self.variables["σ_μ"][0] = self.U(0, np.std(self.train), self.L)

        self.variables["x"][0] = self.N(
            self.variables["x̄"][0], 
            (self.variables["σ_x"][0] ** 2) / (1 - self.variables["ϕ_x"][0] ** 2)
        )
        self.variables["μ"][0] = self.N(
            self.α_μ_parameter, 
            (self.variables["σ_μ"][0] ** 2) / (1 - self.variables["σ_μ"][0] ** 2)
        )   
        self.variables["ϵ"][0] = self.N(0, 1, self.L)
        self.variables["y"][0] = self.observation_function(
            self.variables["μ"][0], self.variables["x"][0], self.variables["ϵ"][0]
        )
    def μ_transition_function(y_t, ϕ_μ, μ_t):
        particles = y_t + ϕ_μ * (μ_t - y_t)
        return particles

    def particle_filtering(self):
        """
        Particle filtering process for state estimation.
        """
        for t in tqdm(range(1, self.T)):
            # SOSS update for prior estimates
            self.variables["p_x̄"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["x̄"][t - 1])
            self.variables["p_ϕ_x"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["ϕ_x"][t - 1])
            self.variables["p_σ_x"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["σ_x"][t - 1])
            self.variables["p_σ_μ"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["σ_μ"][t - 1])
            self.variables["p_ϕ_μ"][t] = self.SOSS_update(self.α_shrink_factor, self.variables["ϕ_μ"][t - 1])

            # Generate system noise and calculate particles for x
            self.variables["ξ"][t] = self.N(0, 1, self.L)
            self.variables["p_x"][t] = self.x_transition_function(
                self.variables["p_x̄"][t],
                self.variables["p_ϕ_x"][t],
                self.variables["p_σ_x"][t],
                self.variables["ξ"][t],
                self.variables["x"][t - 1],
            )

            # Generate system noise and calculate particles for μ
            self.variables["η"][t] = self.N(0, 1, self.L)
            self.variables["p_μ"][t] = self.μ_transition_function(
                self.observation[t - 1],
                self.variables["p_ϕ_μ"][t],
                self.variables["p_σ_μ"][t],
                self.variables["η"][t],
                self.variables["μ"][t - 1],
            )

            # Generate observation noise and calculate expected observations
            self.variables["ϵ"][t] = self.N(0, 1, self.L)
            self.variables["y"][t] = self.observation_function(
                self.variables["p_μ"][t],
                self.variables["p_x"][t],
                self.variables["ϵ"][t],
            )

            # Calculate likelihood using Gaussian observation likelihood
            likelihood = self.observation_likelihood(self.observation[t], self.variables["p_μ"][t], self.variables["p_x"][t])
            self.variables["α"][t] = likelihood

            # Normalize the likelihood to calculate weights
            weights = self.variables["α"][t]
            weights /= np.sum(weights)

            # Resample particles based on weights
            cumulative_sum = np.cumsum(weights)
            positions = (np.arange(self.L) + np.random.uniform(0, 1)) / self.L
            sample_indices = np.searchsorted(cumulative_sum, positions)

            # Resample all variables
            self.variables["x"][t] = self.variables["p_x"][t, sample_indices]
            self.variables["μ"][t] = self.variables["p_μ"][t, sample_indices]
            self.variables["x̄"][t] = self.variables["p_x̄"][t, sample_indices]
            self.variables["ϕ_x"][t] = self.variables["p_ϕ_x"][t, sample_indices]
            self.variables["σ_x"][t] = self.variables["p_σ_x"][t, sample_indices]
            self.variables["ϕ_μ"][t] = self.variables["p_ϕ_μ"][t, sample_indices]
            self.variables["σ_μ"][t] = self.variables["p_σ_μ"][t, sample_indices]
    
    def plot(self):
        """
        Plots the sequential estimations of parameters and expected returns.
        """
        # Compute median, lower, and upper bounds for each parameter
        x̄_median, x̄_lower, x̄_upper = self.compute_stats(self.variables["x̄"])
        ϕ_μ_median, ϕ_μ_lower, ϕ_μ_upper = self.compute_stats(self.variables["ϕ_μ"])
        ϕ_x_median, ϕ_x_lower, ϕ_x_upper = self.compute_stats(self.variables["ϕ_x"])
        σ_x_median, σ_x_lower, σ_x_upper = self.compute_stats(self.variables["σ_x"])
        σ_μ_median, σ_μ_lower, σ_μ_upper = self.compute_stats(self.variables["σ_μ"])

        # Set up plot style
        sns.set_theme(style="whitegrid")  # Use white grid style for better visualization
        plt.rcParams["axes.grid"] = True
        plt.rcParams["axes.grid.axis"] = "y"  # Enable horizontal grid lines only

        # Create the figure and axes for a 3x2 grid of subplots
        fig, axes = plt.subplots(3, 2, figsize=(12, 10))
        dates = self.returns.index  # Dates for x-axis, assuming self.returns contains the returns DataFrame

        # Sequential estimations of x̄
        axes[0, 0].plot(dates, x̄_median, label="Median", color="cornflowerblue")
        axes[0, 0].plot(dates, x̄_lower, linestyle="dotted", color="orange")
        axes[0, 0].plot(dates, x̄_upper, linestyle="dotted", color="orange")
        axes[0, 0].set_title("Sequential estimations of parameter $\overline{x}$")
        axes[0, 0].set_ylim(-2, 6)

        # Sequential estimations of ϕ_μ
        axes[0, 1].plot(dates, ϕ_μ_median, label="Median", color="cornflowerblue")
        axes[0, 1].plot(dates, ϕ_μ_lower, linestyle="dotted", color="orange")
        axes[0, 1].plot(dates, ϕ_μ_upper, linestyle="dotted", color="orange")
        axes[0, 1].set_title("Sequential estimations of parameter $\phi_\mu$")
        axes[0, 1].set_ylim(0, 1.2)

        # Sequential estimations of ϕ_x
        axes[1, 0].plot(dates, ϕ_x_median, label="Median", color="cornflowerblue")
        axes[1, 0].plot(dates, ϕ_x_lower, linestyle="dotted", color="orange")
        axes[1, 0].plot(dates, ϕ_x_upper, linestyle="dotted", color="orange")
        axes[1, 0].set_title("Sequential estimations of parameter $\phi_x$")
        axes[1, 0].set_ylim(0.4, 1.1)

        # Sequential estimations of σ_μ
        axes[1, 1].plot(dates, σ_μ_median, label="Median", color="cornflowerblue")
        axes[1, 1].plot(dates, σ_μ_lower, linestyle="dotted", color="orange")
        axes[1, 1].plot(dates, σ_μ_upper, linestyle="dotted", color="orange")
        axes[1, 1].set_title("Sequential estimations of parameter $\sigma_\mu$")
        axes[1, 1].set_ylim(0, 4.5)

        # Sequential estimations of σ_x
        axes[2, 0].plot(dates, σ_x_median, label="Median", color="cornflowerblue")
        axes[2, 0].plot(dates, σ_x_lower, linestyle="dotted", color="orange")
        axes[2, 0].plot(dates, σ_x_upper, linestyle="dotted", color="orange")
        axes[2, 0].set_title("Sequential estimations of parameter $\sigma_x$")
        axes[2, 0].set_ylim(0, 2.5)

        # Expected returns
        observation = self.observation  # Assuming self.observation contains the observed returns
        y_median = np.median(self.variables["y"], axis=1)  # Median of expected returns
        axes[2, 1].plot(dates[48:], observation[48:], linestyle="dotted", color="grey", label="Observed")
        axes[2, 1].plot(dates[48:], y_median[48:], color="orange", label="Expected")
        axes[2, 1].set_title("Expected Returns")
        axes[2, 1].set_ylim(-20, 15)

        # Add legends to the first subplot in each row
        for i in range(3):
            axes[i, 0].legend()

        # Adjust layout for better appearance
        plt.tight_layout()
        plt.show()


In [35]:
def get_model(model_name, observation, num_particles, time_steps, **kwargs):
    """
    Factory function to get the appropriate model.
    """
    models = {
        "CMSV": CMSVModel(observation, num_particles, time_steps, **kwargs),
        "SMSV": SMSVModel(observation, num_particles, time_steps, **kwargs),
        "SMSV+EMA": SMSV_EMAModel(observation, num_particles, time_steps, **kwargs),
        "SMSV+EMASO": SMSV_EMASOModel(observation, num_particles, time_steps, **kwargs),
    }
    if model_name not in models:
        raise ValueError(f"Model '{model_name}' is not recognized.")
    return models[model_name]


In [38]:
tickers = [
    "^N225",    # S&P 500 Index
      # iShares U.S. Real Estate ETF (Morgan Stanley REIT Index)
]

# Download monthly returns data for the last 14 years
start_date = "2003-03-01"
end_date = "2016-04-01"

# Fetch monthly data for each ticker
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, interval='1mo', progress=False)['Adj Close']
    returns = data.pct_change().dropna() * 100  # Calculate monthly returns

model = get_model(SMSV_EMASOModel, returns, num_particles = 1_000_000, time_steps = 155 )

KeyError: 'ϕ_μ'